In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime

from dateutil.parser import parse

pd.set_option('display.max_rows', 1000)

In [14]:
bike_path = os.path.join('..', '..', 'data', 'raw', '2017_Toronto_Bikeshare.csv')
bike_path

'../../data/raw/2017_Toronto_Bikeshare.csv'

In [15]:
df_raw = pd.read_csv(bike_path)

In [27]:
df_rand = df_raw.sample( n = 1000, random_state = 1 ).copy()
df_rand.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
1335740,2200408,2011-06-17 9:22,2011-06-17 9:27,301.0,Lower Spadina Ave / Lake Shore Blvd,York St / Queens Quay W,Member,NaN,NaN
38449,757907,30/01/2017 13:55,30/01/2017 14:02,410.0,7000,Fort York Blvd / Capreol Crt,7053,Metro Hall Plaza,Member
947287,1776636,9/7/2017 17:32,9/7/2017 17:37,326.0,Richmond St E / Jarvis St Green P,Toronto Eaton Centre (Yonge St),Member,NaN,NaN
1187003,2034426,2010-10-17 7:55,2010-10-17 8:00,305.0,York St / Queens Quay W,Dockside Dr / Queens Quay E (Sugar Beach),Member,NaN,NaN
810518,1628900,8/20/2017 10:24,8/20/2017 12:47,8534.0,King St E / River St,King St E / River St,Member,NaN,NaN


In [35]:
df_rand = df_rand.sort_index(ascending=True)
df_rdate = df_rand[['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds']]
df_rdate.head(10)

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds
2802,715741,4/1/2017 16:05,4/1/2017 16:12,434.0
4352,717690,5/1/2017 20:57,5/1/2017 21:01,264.0
5122,718611,6/1/2017 13:53,6/1/2017 14:05,679.0
8642,722809,10/1/2017 14:30,10/1/2017 14:49,1113.0
8691,722889,10/1/2017 16:26,10/1/2017 16:33,447.0
9985,724501,11/1/2017 21:02,11/1/2017 21:18,978.0
10584,725133,12/1/2017 0:05,12/1/2017 0:10,286.0
10876,725460,12/1/2017 7:46,12/1/2017 8:00,839.0
14692,730005,14/01/2017 19:30,14/01/2017 19:35,310.0
15556,731051,15/01/2017 21:18,15/01/2017 21:21,169.0


#### m: 1-6 seem to be d/m/y
#### m:7-9 seem to be m/d/y
#### m:10 has m/d/y but some mixed in with 2010-xx-17
#### m:11 had m/d/y but some mixed in with 2011-xx-17
#### m:12 has mix of m/d/y but some with 2012-xx-17
#### first m/d/y is loc 465476

In [31]:
df_raw.iloc[465475:465480,:]

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
465475,1253144,30/06/2017 23:59,1/7/2017 0:26,1623.0,7006,Bay St / College St (East Side),7093,Danforth Ave / Gough Ave,Member
465476,1253914,7/1/2017 0:00,7/1/2017 0:15,910.0,Princess St / Adelaide St E,424 Wellington St W,Member,NaN,NaN
465477,1253915,7/1/2017 0:01,7/1/2017 0:15,837.0,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual,NaN,NaN
465478,1253916,7/1/2017 0:01,7/1/2017 0:14,786.0,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual,NaN,NaN
465479,1253917,7/1/2017 0:01,7/1/2017 0:25,1420.0,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual,NaN,NaN


In [18]:
print(df_raw.shape)
print(df_rdate.shape)

(1492370, 9)
(100000, 4)


In [56]:
good_dtm = df_raw.iloc[465475,2]
print(type(good_dtm))
print(good_dtm)


<class 'str'>
1/7/2017 0:26


In [92]:
slash_date = r'\d?\d/\d?\d/(\d{1,2})?1(7|8)'
dash_date = r'\d{1,4}-\d?\d-1(7|8)'
z = re.search(slash_date, '01/07/17 0:26')
if z:
    print((z.group()))
else:
    print('no match')

01/07/17


In [93]:
bad_dtm = df_raw.iloc[1187003,2]
print(type(bad_dtm))
print(bad_dtm)

<class 'str'>
2010-10-17 8:00


In [94]:
z = re.search(dash_date, bad_dtm)
if z:
    print(z.group())
else:
    print('no match')

2010-10-17


In [87]:
string = z.group()
string[1]

'0'

In [96]:
def dash_matcher(date):
    z = re.search(dash_date, date)
    if z:
        return True
    return False

In [97]:
def slash_matcher(date):
    z = re.search(slash_date, date)
    if z:
        return True
    return False

In [106]:
df_rand['dash_reg_flag'] =  df_rand['trip_start_time'].apply(dash_matcher)
df_rand['slash_reg_flag'] = df_rand['trip_start_time'].apply(slash_matcher)
df_rand['date_flag'] = df_rand['dash_reg_flag'] | df_rand['slash_reg_flag']
df_rand.tail(10)

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type,dash_reg_flag,slash_reg_flag,date_flag
1468113,2352440,2012-11-17 19:40,2012-11-17 19:49,514.0,Bloor GO / UP Station (West Toronto Railpath),Wright / Sorauren (Sorauren Park) - SMART,Member,NaN,NaN,True,False,True
1470175,2354997,12/13/17 08:59:05,12/13/17 09:06:45,460.0,Bay St / Wellesley St W,Dundas St W / Yonge St,Member,NaN,NaN,False,True,True
1474341,2360040,12/15/17 09:55:21,12/15/17 10:03:25,484.0,Augusta Ave / Denison Sq,John St / Mercer St - SMART,Member,NaN,NaN,False,True,True
1475260,2361409,12/15/17 19:46:33,12/15/17 19:57:21,648.0,Nelson St / Duncan St,University Ave / Gerrard St W,Member,NaN,NaN,False,True,True
1478946,2366179,12/18/17 22:28:34,12/18/17 22:40:13,699.0,Phoebe St / Spadina Ave,Harbord St / Clinton St,Member,NaN,NaN,False,True,True
1485409,2373943,12/21/17 07:34:54,12/21/17 07:42:31,457.0,College St / Crawford St,Dufferin St / Sylvan Av (Dufferin Grove Park),Member,NaN,NaN,False,True,True
1486199,2375056,12/21/17 11:37:31,12/21/17 11:44:30,419.0,Wellington St W / Portland St,University Ave / King St W,Member,NaN,NaN,False,True,True
1489251,2379153,12/24/17 13:19:56,12/24/17 13:24:31,275.0,Fort York Blvd / Capreol Crt,Bathurst St / Queens Quay W,Member,NaN,NaN,False,True,True
1489489,2379398,12/25/17 11:11:10,12/25/17 11:15:54,284.0,Dundas St W / Yonge St,Dundas St W / Yonge St,Casual,NaN,NaN,False,True,True
1491737,2382766,12/30/17 12:57:46,12/30/17 13:05:25,459.0,Bay St / Wellesley St W,College St / Huron St,Member,NaN,NaN,False,True,True


In [111]:
df_rand['date_flag'].value_counts()

True    1000
Name: date_flag, dtype: int64

In [19]:
date_series_1 = df_rdate['trip_start_time'].str.split(expand = True)
df_rdate['start_date'] = date_series_1[0]
df_rdate['start_time'] = date_series_1[1]
date_series_2 = df_rdate['trip_stop_time'].str.split(expand = True)
df_rdate['end_date'] = date_series_2[0]
df_rdate['end_time'] = date_series_2[1]

/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a

In [20]:
df_rdate

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,start_date,start_time,end_date,end_time
5,712387,1/1/2017 0:12,1/1/2017 0:22,571.0,1/1/2017,0:12,1/1/2017,0:22
21,712403,1/1/2017 1:12,1/1/2017 1:19,398.0,1/1/2017,1:12,1/1/2017,1:19
24,712406,1/1/2017 1:31,1/1/2017 1:34,182.0,1/1/2017,1:31,1/1/2017,1:34
27,712409,1/1/2017 1:32,1/1/2017 1:40,474.0,1/1/2017,1:32,1/1/2017,1:40
31,712413,1/1/2017 1:50,1/1/2017 2:06,955.0,1/1/2017,1:50,1/1/2017,2:06
...,...,...,...,...,...,...,...,...
1492312,2383590,12/31/17 18:59:48,12/31/17 19:07:53,485.0,12/31/17,18:59:48,12/31/17,19:07:53
1492324,2383602,12/31/17 19:52:42,12/31/17 19:58:31,349.0,12/31/17,19:52:42,12/31/17,19:58:31
1492325,2383603,12/31/17 20:02:04,12/31/17 20:05:24,200.0,12/31/17,20:02:04,12/31/17,20:05:24
1492346,2383624,12/31/17 21:46:49,12/31/17 21:52:14,325.0,12/31/17,21:46:49,12/31/17,21:52:14


In [21]:
date_series_1 = df_rdate['start_date'].str.split(expand = True)
df_rdate['start_date'] = date_series_1[0]
df_rdate['start_time'] = date_series_1[1]
date_series_2 = df_rdate['stop_date'].str.split(expand = True)
df_rdate['end_date'] = date_series_2[0]
df_rdate['end_time'] = date_series_2[1]

/Users/brycehaley/anaconda3/envs/1000-p1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyError: 1

In [124]:
def take_list(lst: list):
    for x in lst:
        print(x)

In [118]:
type(['string1', 'string2'])

list

In [125]:
take_list(['string1', 'string2'])

string1
string2


In [133]:
def mutate_df(df):
    df['new'] = df['col1'] + df['col2']
    return df

In [134]:
df = pd.DataFrame(data = {'col1': [1, 2], 'col2': [3, 4]})

In [135]:
df = mutate_df(df)
df

,col1,col2,new
0,1,3,4
1,2,4,6


In [130]:
df

,col1,col2
0,1,3
1,2,4
